In [1]:
import os
import random
import copy
import cv2
import timm
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

from albumentations.pytorch import ToTensorV2
import albumentations as A
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
from timm.loss import LabelSmoothingCrossEntropy




In [6]:
# ============================================================
# 0️⃣ 전역 설정 (seed, path, config)
# ============================================================
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

BASE_DIR = "/data/ephemeral/home/data"
TRAIN_META = os.path.join(BASE_DIR, "meta_stage0_10_1_train_v10.csv")
TEST_DIR = os.path.join(BASE_DIR, "processed", "stage0_10_1_test_v10")
SAMPLE_SUB = os.path.join(BASE_DIR, "raw", "sample_submission.csv")
OUT_DIR = "./runs_v10_b4_b5"
os.makedirs(OUT_DIR, exist_ok=True)

SPECIAL_CLASSES = {3, 7, 14}

# --- EfficientNet-B4 (main) ---
CFG_MAIN = dict(
    model_name="tf_efficientnet_b4_ns",
    num_classes=17,
    img_size=380,
    batch_size=32,      # 3090 기준 적절
    lr=3e-4,
    weight_decay=1e-4,
    epochs=50,
    patience=10,
    valid_ratio=0.8,
    num_workers=4,
    tta_times=20,
)

# --- EfficientNet-B5 (sub, class 3/7/14 전용) ---
CFG_SUB = dict(
    model_name="efficientnet_b5.sw_in12k_ft_in1k",
    num_classes=3,
    img_size=448,
    batch_size=16,
    lr=3e-4,
    weight_decay=1e-4,
    epochs=40,
    patience=10,
    valid_ratio=0.8,
    num_workers=4,
    tta_times=10,
    target_classes=sorted(list(SPECIAL_CLASSES)),
)


# ============================================================
# 1️⃣ Dataset 정의 (oversample, special-class control)
# ============================================================
class V10ImageDataset(Dataset):
    def __init__(self, csv_path, transform=None, oversample=False,
                 filter_special=False, for_sub=False, special_classes=SPECIAL_CLASSES):
        df = pd.read_csv(csv_path)

        # 🔹 특정 클래스만 학습할 경우 (3, 7, 14)
        if filter_special:
            df = df[df["target"].isin(special_classes)].reset_index(drop=True)

        # 🔹 클래스 불균형 보정 (oversample)
        if oversample:
            df = self._oversample(df)

        # 🔹 b5 서브모델용 클래스 매핑
        if for_sub:
            mapping = {c: i for i, c in enumerate(sorted(list(special_classes)))}
            df["sub_target"] = df["target"].map(mapping)
            self.use_sub_target = True
        else:
            self.use_sub_target = False

        self.df = df
        self.transform = transform

    def _oversample(self, df):
        """클래스 3, 7, 14 비율 증가"""
        factors = {3: 2, 7: 2, 14: 3}
        parts = [df]
        for cls, f in factors.items():
            sub = df[df["target"] == cls]
            if len(sub) > 0 and f > 1:
                parts.append(sub.loc[sub.index.repeat(f - 1)])
        out = pd.concat(parts, axis=0).reset_index(drop=True)
        print("✅ Oversample 완료:", {k: (out['target'] == k).sum() for k in factors})
        return out

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = cv2.imread(row["filepath"])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = row["sub_target"] if self.use_sub_target else row["target"]
        if self.transform:
            img = self.transform(image=img)["image"]
        return img, int(label)


# ============================================================
# 2️⃣ Test Dataset 정의
# ============================================================
class TestImageDataset(Dataset):
    def __init__(self, sample_csv, test_dir, transform):
        # ✅ 문자열 경로(str) 또는 DataFrame 둘 다 허용
        if isinstance(sample_csv, str):
            self.df = pd.read_csv(sample_csv)
        elif isinstance(sample_csv, pd.DataFrame):
            self.df = sample_csv.copy()
        else:
            raise ValueError("❌ sample_csv must be a path or a DataFrame")

        self.test_dir = test_dir
        self.transform = transform
        self.df["ID"] = self.df["ID"].apply(lambda x: x if str(x).endswith(".jpg") else f"{x}.jpg")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df.iloc[idx]["ID"]
        path = os.path.join(self.test_dir, img_id)
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transform(image=img)["image"]
        return img, img_id


In [8]:

# ============================================================
# 2️⃣ Transform
# ============================================================
def get_transform(img_size, tta=False):
    base = [
        A.LongestMaxSize(max_size=img_size, always_apply=True),
        A.PadIfNeeded(
            min_height=img_size, min_width=img_size,
            border_mode=cv2.BORDER_CONSTANT, value=(255, 255, 255)
        ),
        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
    if not tta:
        return A.Compose([
            A.RandomRotate90(p=0.5),
            A.OneOf([
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
            ], p=0.5),
            A.RandomBrightnessContrast(p=0.1),
            # ✅ 최신 버전용 Cutout 대체
            A.CoarseDropout(
                max_holes=6,
                max_height=img_size // 10,
                max_width=img_size // 10,
                min_holes=1,
                min_height=img_size // 20,
                min_width=img_size // 20,
                fill_value=255,
                mask_fill_value=None,
                p=0.3
            ),
        ] + base)
    else:
        return A.Compose([
            A.RandomRotate90(p=1.0),
            A.OneOf([
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
            ], p=0.5),
        ] + base)

# ============================================================
# 3️⃣ 학습 함수
# ============================================================
def create_model(model_name, num_classes, drop=0.2):
    model = timm.create_model(model_name, pretrained=True, num_classes=num_classes, drop_rate=drop)
    model.to(device)
    return model

def train_one_epoch(model, loader, criterion, optimizer, scaler):
    model.train(); total_loss = 0; preds, tgts = [], []
    for imgs, lbls in tqdm(loader, desc="[Train]", leave=False):
        imgs, lbls = imgs.to(device), lbls.to(device)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(True):
            out = model(imgs); loss = criterion(out, lbls)
        scaler.scale(loss).backward(); scaler.step(optimizer); scaler.update()
        total_loss += loss.item()
        preds += out.argmax(1).cpu().tolist()
        tgts += lbls.cpu().tolist()
    return total_loss/len(loader), accuracy_score(tgts,preds), f1_score(tgts,preds,average='macro')

@torch.no_grad()
def eval_one_epoch(model, loader, criterion):
    model.eval(); total_loss = 0; preds, tgts = [], []
    for imgs, lbls in tqdm(loader, desc="[Valid]", leave=False):
        imgs, lbls = imgs.to(device), lbls.to(device)
        with torch.cuda.amp.autocast(True):
            out = model(imgs); loss = criterion(out, lbls)
        total_loss += loss.item()
        preds += out.argmax(1).cpu().tolist()
        tgts += lbls.cpu().tolist()
    return total_loss/len(loader), accuracy_score(tgts,preds), f1_score(tgts,preds,average='macro')

@torch.no_grad()
def infer_probs(model, loader):
    model.eval(); all_probs, all_ids = [], []
    for imgs, ids in tqdm(loader, desc="[Infer]", leave=False):
        imgs = imgs.to(device)
        with torch.cuda.amp.autocast(True):
            out = F.softmax(model(imgs), dim=1)
        all_probs.append(out.cpu().numpy()); all_ids.extend(ids)
    return np.concatenate(all_probs), all_ids

# ============================================================
# 4️⃣ 학습 루프 (공용)
# ============================================================
def train_loop(cfg, train_ds, valid_ds, model_name, num_classes, model_tag):
    train_loader = DataLoader(train_ds, batch_size=cfg["batch_size"], shuffle=True, num_workers=cfg["num_workers"], pin_memory=True)
    valid_loader = DataLoader(valid_ds, batch_size=cfg["batch_size"], shuffle=False, num_workers=cfg["num_workers"], pin_memory=True)

    model = create_model(model_name, num_classes)
    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg["lr"], weight_decay=cfg["weight_decay"])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=cfg["epochs"], eta_min=1e-6)
    criterion = LabelSmoothingCrossEntropy(smoothing=0.05)
    scaler = torch.cuda.amp.GradScaler()

    best_f1, patience, counter = -1, cfg["patience"], 0
    best_path = os.path.join(OUT_DIR, f"model_{model_tag}_best.pt")

    for e in range(1, cfg["epochs"]+1):
        print(f"\n==== [{model_tag}] Epoch {e}/{cfg['epochs']} ====")
        tr_l,tr_a,tr_f=train_one_epoch(model,train_loader,criterion,optimizer,scaler)
        va_l,va_a,va_f=eval_one_epoch(model,valid_loader,criterion)
        scheduler.step()
        print(f"[Train] loss={tr_l:.4f} acc={tr_a:.4f} f1={tr_f:.4f}")
        print(f"[Valid] loss={va_l:.4f} acc={va_a:.4f} f1={va_f:.4f}")
        if va_f>best_f1:
            best_f1=va_f; counter=0
            torch.save(model.state_dict(),best_path)
            print(f"✅ Best updated: F1={best_f1:.5f}")
        else:
            counter+=1; print(f"⏳ No improve ({counter}/{patience})")
            if counter>=patience: print("🛑 Early stop"); break
    return best_path, best_f1

# ============================================================
# 5️⃣ Main / Sub 학습 & 2-stage inference
# ============================================================
def train_main():
    full = V10ImageDataset(TRAIN_META, get_transform(CFG_MAIN["img_size"]), oversample=True)
    n = int(len(full) * (1 - CFG_MAIN["valid_ratio"]))  # ✅ 수정
    tr, v = random_split(full, [len(full) - n, n], generator=torch.Generator().manual_seed(SEED))
    v = copy.deepcopy(v); v.dataset.transform = get_transform(CFG_MAIN["img_size"], tta=False)
    return train_loop(CFG_MAIN, tr, v, CFG_MAIN["model_name"], CFG_MAIN["num_classes"], "main_b4")

def train_sub():
    full = V10ImageDataset(TRAIN_META, get_transform(CFG_SUB["img_size"]), filter_special=True, for_sub=True)
    n = int(len(full) * (1 - CFG_SUB["valid_ratio"]))  # ✅ 수정
    tr, v = random_split(full, [len(full) - n, n], generator=torch.Generator().manual_seed(SEED))
    v = copy.deepcopy(v); v.dataset.transform = get_transform(CFG_SUB["img_size"], tta=False)
    return train_loop(CFG_SUB, tr, v, CFG_SUB["model_name"], CFG_SUB["num_classes"], "sub_b5")


@torch.no_grad()
def two_stage_infer(main_ckpt, sub_ckpt):
    print("\n🔄 2-Stage Inference 시작")
    main=create_model(CFG_MAIN["model_name"],CFG_MAIN["num_classes"]); main.load_state_dict(torch.load(main_ckpt)); main.eval()
    sub=create_model(CFG_SUB["model_name"],CFG_SUB["num_classes"]); sub.load_state_dict(torch.load(sub_ckpt)); sub.eval()

    base_ds=TestImageDataset(SAMPLE_SUB,TEST_DIR,get_transform(CFG_MAIN["img_size"]))
    base_loader=DataLoader(base_ds,batch_size=CFG_MAIN["batch_size"],shuffle=False)
    main_probs,ids=infer_probs(main,base_loader)
    tta_sum=np.zeros_like(main_probs)
    for i in range(CFG_MAIN["tta_times"]):
        print(f"[TTA] round {i+1}/{CFG_MAIN['tta_times']}")
        tta_ds=TestImageDataset(SAMPLE_SUB,TEST_DIR,get_transform(CFG_MAIN["img_size"],tta=True))
        tta_loader=DataLoader(tta_ds,batch_size=CFG_MAIN["batch_size"],shuffle=False)
        tta_sum+=infer_probs(main,tta_loader)[0]
    final=(main_probs+tta_sum/CFG_MAIN["tta_times"])/2
    preds=final.argmax(1)

    # Sub-model correction for 3/7/14
    sub_map={i:c for i,c in enumerate(CFG_SUB["target_classes"])}
    special_idx=np.where(np.isin(preds,CFG_SUB["target_classes"]))[0]
    if len(special_idx)>0:
        ids_sel=[ids[i] for i in special_idx]
        sub_ds=TestImageDataset(pd.DataFrame({"ID":ids_sel}),TEST_DIR,get_transform(CFG_SUB["img_size"]))
        sub_loader=DataLoader(sub_ds,batch_size=CFG_SUB["batch_size"],shuffle=False)
        sub_probs,ids2=infer_probs(sub,sub_loader)
        sub_preds=np.vectorize(sub_map.get)(sub_probs.argmax(1))
        for i,j in zip(special_idx,sub_preds): preds[i]=j

    out=pd.DataFrame({"ID":ids,"target":preds})
    path=os.path.join(OUT_DIR,"submission_v10_b4_b5.csv")
    out.to_csv(path,index=False)
    print(f"✅ 결과 저장: {path}")


In [4]:

def main():
    main_ckpt,f1m=train_main()
    sub_ckpt,f1s=train_sub()
    print(f"\n[완료] MAIN_F1={f1m:.4f}, SUB_F1={f1s:.4f}")
    two_stage_infer(main_ckpt,sub_ckpt)

if __name__=="__main__":
    main()


/tmp/ipykernel_353749/20435626.py:6: UserWarning: Argument(s) 'always_apply' are not valid for transform MaxSizeTransform
  A.LongestMaxSize(max_size=img_size, always_apply=True),
/tmp/ipykernel_353749/20435626.py:7: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(
/tmp/ipykernel_353749/20435626.py:24: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width, fill_value, mask_fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(
/tmp/ipykernel_353749/20435626.py:6: UserWarning: Argument(s) 'always_apply' are not valid for transform MaxSizeTransform
  A.LongestMaxSize(max_size=img_size, always_apply=True),
/tmp/ipykernel_353749/20435626.py:7: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(
/tmp/ipykernel_353749/20435626.py:24: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width, fill_value, mask_fill_value' are

✅ Oversample 완료: {3: np.int64(12672), 7: np.int64(12928), 14: np.int64(9792)}


/tmp/ipykernel_353749/20435626.py:99: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()



==== [main_b4] Epoch 1/50 ====


[Train]:   0%|                             | 0/2996 [00:00<?, ?it/s]/tmp/ipykernel_353749/20435626.py:58: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(True):
[Valid]:   0%|                              | 0/749 [00:00<?, ?it/s]/tmp/ipykernel_353749/20435626.py:71: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(True):
                                                                    

...

[TTA] round 19/20


[Infer]:   0%|                               | 0/99 [00:00<?, ?it/s]/tmp/ipykernel_353749/20435626.py:83: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(True):
/tmp/ipykernel_353749/20435626.py:6: UserWarning: Argument(s) 'always_apply' are not valid for transform MaxSizeTransform
  A.LongestMaxSize(max_size=img_size, always_apply=True),
/tmp/ipykernel_353749/20435626.py:7: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(


[TTA] round 20/20


[Infer]:   0%|                               | 0/99 [00:00<?, ?it/s]/tmp/ipykernel_353749/20435626.py:83: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(True):
/tmp/ipykernel_353749/20435626.py:6: UserWarning: Argument(s) 'always_apply' are not valid for transform MaxSizeTransform
  A.LongestMaxSize(max_size=img_size, always_apply=True),
/tmp/ipykernel_353749/20435626.py:7: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(
/tmp/ipykernel_353749/20435626.py:24: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width, fill_value, mask_fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(


TypeError: argument of type 'method' is not iterable

In [9]:

two_stage_infer(
    main_ckpt="./runs_v10_b4_b5/model_main_b4_best.pt",
    sub_ckpt="./runs_v10_b4_b5/model_sub_b5_best.pt"
)



🔄 2-Stage Inference 시작


/data/ephemeral/home/py310/lib/python3.10/site-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(
/tmp/ipykernel_353749/2330462946.py:141: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you do

[TTA] round 1/20


[Infer]:   0%|                               | 0/99 [00:00<?, ?it/s]/tmp/ipykernel_353749/2330462946.py:83: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(True):
/tmp/ipykernel_353749/2330462946.py:6: UserWarning: Argument(s) 'always_apply' are not valid for transform MaxSizeTransform
  A.LongestMaxSize(max_size=img_size, always_apply=True),
/tmp/ipykernel_353749/2330462946.py:7: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(


[TTA] round 2/20


...

[TTA] round 19/20


[Infer]:   0%|                               | 0/99 [00:00<?, ?it/s]/tmp/ipykernel_353749/2330462946.py:83: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(True):
/tmp/ipykernel_353749/2330462946.py:6: UserWarning: Argument(s) 'always_apply' are not valid for transform MaxSizeTransform
  A.LongestMaxSize(max_size=img_size, always_apply=True),
/tmp/ipykernel_353749/2330462946.py:7: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(


[TTA] round 20/20


[Infer]:   0%|                               | 0/99 [00:00<?, ?it/s]/tmp/ipykernel_353749/2330462946.py:83: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(True):
/tmp/ipykernel_353749/2330462946.py:6: UserWarning: Argument(s) 'always_apply' are not valid for transform MaxSizeTransform
  A.LongestMaxSize(max_size=img_size, always_apply=True),
/tmp/ipykernel_353749/2330462946.py:7: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(
/tmp/ipykernel_353749/2330462946.py:24: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width, fill_value, mask_fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(
                                                                    

✅ 결과 저장: ./runs_v10_b4_b5/submission_v10_b4_b5.csv
